# Finite Difference Methods
## Introduction

### Definitions
https://en.wikipedia.org/wiki/Numerical_methods_for_partial_differential_equations
#### Partial differential equations (PDEs)
https://en.wikipedia.org/wiki/Partial_differential_equation  
Partial differential equations (PDEs) are equations involving the partial differential of one or more of their terms. Often these partial dertivatives are in terms of space and time. Often these equations are used to model real world phenomena, particularly in continuum mechanics, such as material stresses, body forces, heat flow, electromagnetic fields, etc.

#### Finitie Difference Methods (FDMs)
https://en.wikipedia.org/wiki/Finite_difference_method  
Finitie Difference Methods (FDMs) are numerical methods for solving PDEs that use Taylor expansions of the partial derivatives to make approximations of the solution with a finitely small discretization of domain.

#### Finitie Element Methods (FEMs)
https://en.wikipedia.org/wiki/Finite_element_method  
Finitie Element Methods (FEMs) are similar to FDMs, but generalize to more complex geometry and allow specification of the discretization into a mesh.

### Tools
#### NumPy
https://numpy.org/  
https://en.wikipedia.org/wiki/NumPy  
NumPy is a scientific computing package that is designed to provide a robust n-dimensional array object and a suite of optimized linear algebra algorithms.  
It is the backbone of almost all Python scientific computing packages.  

In [ ]:
import numpy as np
import scipy.ndimage, scipy.linalg

#### Matplotlib
https://matplotlib.org/  
https://en.wikipedia.org/wiki/Matplotlib  
Matplotlib is a plotting package that is designed to provide a suite of standard plotting and visualization tools.  

In [ ]:
# Matplotlib is used to render visualizations of the solution
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
%matplotlib inline

## PDE Problems
### Classification of PDES
#### Order
The order of a PDE is the order of the highest derivative in its expression
$$ F \left ( \vec{x}, u, \frac{\partial u}{\partial \vec{x}}, ..., \frac{\partial^{n-1} u}{\partial \vec{x}^{n-1}}, \frac{\partial^{n} u}{\partial \vec{x}^{n}} \right ) = 0 $$
Where:
- $u$ is the dependant variable (a multi-variate function of $\vec{x}$)
- $\vec{x}$ is the (vector-valued) independant variable
- $\frac{\partial^{i} u}{\partial \vec{x}^{i}}$ is the $i$th order partial derivative of the dependant variable, $u$, with respect to the elements of the independant variable $\vec{x}$
- $n$ is the order of the PDE

#### Linearity
A PDE is **linear** if its expression is linear in the dependant variable and its derivatives.
$$ \sum_{i} \sum_{j} a_{i,j} \frac{\partial^{j} u}{\partial x_{i}^{j}} = f \left ( \vec{x} \right ) $$
Where:
- $u$ is the dependant variable (a multi-variate function of $\vec{x}$)
- $\vec{x}$ is the (vector-valued) independant variable
- $x_{i}$ is the $i$th element of the vector-valued independant variable, $\vec{x}$
- $\frac{\partial^{j} u}{\partial x_{i}^{j}}$ is the $j$th partial derivative of the dependant variable, $u$, with respect to $x_{i}$
- $a_{i,j}$ is a coefficient, which can be a function of the dependant variable $\vec{x}$
- $f \left ( \vec{x} \right )$ is a (multi-variate) function

#### Notable Special Case: Elliptic PDES
https://en.wikipedia.org/wiki/Elliptic_partial_differential_equation  

#### Notable Special Case: Parabolic PDES
https://en.wikipedia.org/wiki/Parabolic_partial_differential_equation  

#### Notable Special Case: Hyperbolic PDES
https://en.wikipedia.org/wiki/Hyperbolic_partial_differential_equation

### Boundary Conditions
Then the boundary conditions further specify the problem
#### Dirichlet
https://en.wikipedia.org/wiki/Dirichlet_boundary_condition  
Fixed-value boundary conditions
$$\forall \vec{x} \in \partial \Omega$$
$$u\left (\vec{x}\right ) = g\left (\vec{x}\right )$$

#### Neumann
https://en.wikipedia.org/wiki/Neumann_boundary_condition  
Fixed-derivative boundary conditions
$$\forall \vec{x} \in \partial \Omega$$
$$\frac{\partial u}{\partial \vec{n}} \left (\vec{x}\right ) = g\left (\vec{x}\right )$$

#### Robin
https://en.wikipedia.org/wiki/Robin_boundary_condition  
A weighted-combination of Dirichlet and Neumann boundary conditions
$$\forall \vec{x} \in \partial \Omega$$
$$a \frac{\partial u}{\partial \vec{n}} \left (\vec{x}\right ) + b u\left (\vec{x}\right )= g\left (\vec{x}\right )$$

#### Cauchy
https://en.wikipedia.org/wiki/Cauchy_boundary_condition  
Both a Dirichlet condition and a Neumann condition (for second order problems)
$$\forall \vec{x} \in \partial \Omega$$
$$u\left (\vec{x}\right ) = g_{1}\left ( \vec{x} \right )$$
and
$$\frac{\partial u}{\partial \vec{n}} \left ( \vec{x} \right ) = g_{2}\left (\vec{x}\right )$$

#### Mixed
Uses Dirichlet and Neumann on different parts of the boundary
$$\Gamma_{1} \cup \Gamma_{2} = \partial \Omega$$
$$\forall \vec{x}_{1} \in \Gamma_{1}$$
$$u\left ( \vec{x}_{1} \right ) = g_{1}\left ( \vec{x}_{1} \right )$$
and
$$\forall \vec{x}_{2} \in \Gamma_{2}$$
$$\frac{\partial u}{\partial \vec{n}} \left ( \vec{x}_{2} \right ) = g_{2}\left ( \vec{x}_{2} \right )$$

## Numerical Solutions to PDEs via FDMs
Finitie Difference Methods (FDMs) are numerical methods for solving PDEs that use Taylor expansions of the partial derivatives to make approximations of the solution with a finitely small discretization of domain.  
$$ x \quad \rightarrow \quad i $$
$$ f \left ( x \right ) \quad \rightarrow \quad f_{i} $$ 
$$ f \left ( x + \mathrm{d} x \right ) \quad \rightarrow \quad f_{i+1} $$ 
$$ f \left ( x - \mathrm{d} x \right ) \quad \rightarrow \quad f_{i-1} $$ 
For **linear** PDEs these equations become a system of linear equations, often of special types, such as **sparse**, or **banded** matrices, for which efficient algorithms exist.

For the next couple of examples we shall use
$$ t \quad \rightarrow \quad i $$
$$ x \quad \rightarrow \quad j $$
$$ u \left ( t, x \right ) \quad \rightarrow \quad u_{i,j} $$ 
$$ f \left ( t, x \right ) \quad \rightarrow \quad f_{i,j} $$ 

### FTCS (Forward-Time Centered-Space)
https://en.wikipedia.org/wiki/FTCS_scheme  
https://en.wikipedia.org/wiki/Euler_method  
Also known as "**Forward-Euler**", as it is essentially the formulation as Euler's Method for Ordinary Differential Equations.
$$ \frac{\partial u}{\partial t} - \alpha \nabla^{2} u = f\left ( t, x \right ) \quad \rightarrow \quad \frac{u_{i+1,j}-u_{i,j}}{\Delta t} - \alpha \frac{u_{i,j+1} + u_{i,j-1} - 2 u_{i,j}}{\Delta x ^{2}} = f_{i,j} $$  
It can be computed efficiently with an iterated loop, as in Euler's Method. However it is only conditionally stable for parabolic PDEs, and unstable for others.

### BTCS (Backward-Time Centered-Space)
https://en.wikipedia.org/wiki/Backward_Euler_method  
Also known as "**Backward-Euler**", as it is essentially the formulation as backwards Euler's Method for Ordinary Differential Equations.
$$ \frac{\partial u}{\partial t} - \alpha \nabla^{2} u = f\left ( t, x \right ) \quad \rightarrow \quad \frac{u_{i+1,j}-u_{i,j}}{\Delta t} - \alpha \frac{u_{i+1,j+1} + u_{i+1,j-1} - 2 u_{i+1,j}}{\Delta x ^{2}} = f_{i+1,j} $$  
It harder to compute efficiently as it requires a full linear system to be set-up. It is however numerically stable.

### Crank-Nicholson
https://en.wikipedia.org/wiki/Crank%E2%80%93Nicolson_method  
https://en.wikipedia.org/wiki/Trapezoidal_rule_(differential_equations)  
Also known as "**Trapezoidal**", as it is essentially the formulation as trapezoidal rule for Ordinary Differential Equations.
$$ \frac{\partial u}{\partial t} - \alpha \nabla^{2} u = f\left ( t, x \right ) \quad \rightarrow \quad \frac{u_{i+1,j}-u_{i,j}}{\Delta t} - \frac{1}{2} \left ( \alpha \frac{u_{i+1,j+1} + u_{i+1,j-1} - 2 u_{i+1,j}}{\Delta x ^{2}} + \alpha \frac{u_{i,j+1} + u_{i,j-1} - 2 u_{i,j}}{\Delta x ^{2}}\right )= \frac{1}{2} \left ( f_{i+1,j} + f_{i,j} \right )$$  
It is increasingly harder to compute efficiently as it requires a full linear system to be set-up. It is however numerically stable, and more accurate than BTCS.

## A Simple Example: The Heat Equation
### Problem Definition
Let's study a simple example of the heat equation with constant boundaries.
- **class**: Linear 2nd order, parabolic (i.e. the heat equation)
- **dimension**: 2 (1 time + 1 space)
- **boundaries**: Dirichlet (IVP in time, fixed boundary in space)

The problem can therefore be written:
$$ \frac{\partial u}{\partial t} - \alpha  \frac{\partial^2 u}{\partial x^2} = f\left ( t, x \right ) $$

Subject to
$$ u\left ( 0, x \right ) = g_{0} \left ( x \right ) \\
u\left ( t, x_{1} \right ) = h_{1} \\
u\left ( t, x_{2} \right ) = h_{2} $$

### Solution Method
Forward-Euler will be used for this numerical solution/simulation.  
#### Discretization
$$ \begin{align}
t && \rightarrow && i \\
x && \rightarrow && j \\
u\left ( t, x \right ) && \rightarrow && u_{i,j} \\
f\left ( t, x \right ) && \rightarrow && f_{i,j}
\end{align}$$

$$ \begin{align}
\frac{\partial u}{\partial t} - \alpha  \frac{\partial^2 u}{\partial x^2} = f\left ( t, x \right ) && \rightarrow && \frac{u_{i+1,j}-u_{i,j}}{\Delta t} - \alpha \frac{u_{i,j+1} + u_{i,j-1} - 2 u_{i,j}}{\Delta x ^{2}} = f_{i,j}
\end{align}$$

Isolating we obtain
$$ \begin{align}
u_{i+1,j} = u_{i,j} + \frac{\alpha \Delta t }{\Delta x ^{2}} \left ( u_{i,j+1} + u_{i,j-1} - 2 u_{i,j} \right ) + \Delta t f_{i,j} 
&& \forall (i,j) \in [1,N_{t}-1] \times [1,N_{x}-1] \\
u_{0,j} = g_{0,j} 
&& \forall j \in [0,N_{x}] \\
u_{i,0} = h_{1} 
&& \forall i \in [1,N_{t}-1] \\
u_{i,N-1} = h_{2} 
&& \forall i \in [1,N_{t}-1]
\end{align}$$

#### Simulation Parameters
Edit these!

In [ ]:
# Define the source term
f = lambda t,x: 0*x*t
a = 0.1

# Set up domain parameters
x_1 = -3
x_2 = 3

t_f = 20

# Boundary conditions
h_1 = 0
h_2 = 0
g_0 = lambda x: np.sin(2*np.pi * x / (x_2 - x_1))
#g_0 = lambda x: np.exp(-(2*np.pi*x / (x_2 - x_1))**2)

In [ ]:
# Set up mesh discretization parameters
N_x = 20+1
N_t = 100

#### Set Up and Solving

In [ ]:
# Create the grid
dx = (x_2 - x_1) / float(N_x)
dt = t_f / float(N_t)

# Construct discrete domain
x = np.linspace(x_1, x_2, N_x)
t = np.linspace(0, t_f, N_t)

# Initialize the discrete solution
u = np.zeros((N_t, N_x))

# Create the discretized source term
#f_ij = np.fromfunction(lambda i,j: f(t[i],x[j]),(N_t,N_x), dtype=int)

# Fill in the boundary conditions
# IVP
u[0,:] = g_0(x)
# Boundaries
u[:,0] = h_1
u[:,-1] = h_2

In [ ]:
# Perform a simple forward-Euler iterative loop to solve
for i in range(N_t-1):
    u[i+1,1:-1] = u[i,1:-1] + (a*dt/dx/dx) * (u[i,:-2] + u[i,2:] - 2*u[i,1:-1]) + dt*f(t[i],x[1:-1])

#### Visualize the Result

In [ ]:
fig, ax = plt.subplots()
ln, = plt.plot(x, u[0,:], animated=True)

def init():
    ln.set_data(x,u[0,:])
    return ln,

def update(frame_n):
    ln.set_data(x, u[frame_n,:])
    return ln,

t_anim = 5
anim = FuncAnimation(fig, update, frames=N_t, init_func=init, blit=True, interval = t_anim*1000/N_t, repeat=True)
plt.close()
HTML(anim.to_jshtml())

#### Numerical Stability
https://en.wikipedia.org/wiki/Von_Neumann_stability_analysis  
The above FTCS is only stable for
$$ \frac{\alpha \Delta t}{\left ( \Delta x\right )^{2}} \leq \frac{1}{2}$$
Try changing the values in the above example and rerunning it!

## A Further Example: The Wave Equation
### Problem Definition
Let's study a simple example of the wave equation with constant boundaries.
- **class**: Linear 2nd order, Hyperbolic (i.e. the wave equation)
- **dimension**: 2 (1 time + 1 space)
- **boundaries**: Mixed (Cauchy IVP in time, Dirichlet fixed boundary in space)

The problem can therefore be written:
$$ \frac{\partial^2 u}{\partial t^2} - c^{2} \frac{\partial^2 u}{\partial x^2} = f\left ( t, x \right ) $$
Subject to
$$ u\left ( 0, x \right ) = g_{0} \left ( x \right ) \\
\frac{\partial u}{\partial t} \left ( 0, x \right ) = g_{1} \left ( x \right ) \\
u\left ( t, x_{1} \right ) = h_{1} \\
u\left ( t, x_{2} \right ) = h_{2} $$

### Solution Method
CTCS will work fine for this solution 
#### Discretization
$$ \begin{align}
t && \rightarrow && i \\
x && \rightarrow && j \\
u\left ( t, x \right ) && \rightarrow && u_{i,j} \\
f\left ( t, x \right ) && \rightarrow && f_{i,j}
\end{align}$$

$$ \begin{align}
\frac{\partial u}{\partial t} - c^{2} \frac{\partial^2 u}{\partial x^2} = f\left ( t, x \right ) 
&& \rightarrow &&
\frac{u_{i+1,j} + u_{i-1,j} - 2 u_{i,j}}{\Delta t} - c^{2} \frac{u_{i,j+1} + u_{i,j-1} - 2 u_{i,j}}{\Delta x ^{2}} = f_{i,j}
\end{align}$$

Isolating we obtain
$$ \begin{align}
\frac{u_{i+1,j} + u_{i-1,j} - 2 u_{i,j}}{\Delta t} - c^{2} \frac{u_{i,j+1} + u_{i,j-1} - 2 u_{i,j}}{\Delta x ^{2}} = f_{i,j} 
&& \forall (i,j) \in [1,N_{t}-2] \times [1,N_{x}-1] \\
u_{0,j} = g_{0,j}
&& \forall j \in [0,N_{x}] \\
\frac{u_{0+1,j} - u_{0,j}}{\Delta t} = g_{1,j}
&& \forall j \in [0,N_{x}] \\
u_{i,0} = h_{1}
&& \forall i \in [1,N_{t}-1] \\
u_{i,N-1} = h_{2}
&& \forall i \in [1,N_{t}-1]
\end{align}$$

#### Simulation Parameters
Edit these!

In [ ]:
# Define the source term
f = lambda t,x: 0*x*t
c = 0.4

# Set up domain parameters
x_1 = -3
x_2 = 3

t_f = 3

# Boundary conditions
h_1 = 0
h_2 = 0
g_0 = lambda x: np.sin(2*np.pi * x / (x_2 - x_1))
g_1 = lambda x: 0*x

In [ ]:
# Set up mesh discretization parameters
N_x = 20+1
N_t = 100

#### Set Up and Solving

In [ ]:
# Initialize the equations and vector
col = []
eqs = []

In [ ]:
# differential equation definition
for i in range(1,N_t-1):
    for j in range(1,N_x-1):
        mat = np.zeros((N_t, N_x))
        # time difference
        mat[i,j] += -2/dt/dt
        mat[i-1,j] += 1/dt/dt
        mat[i+1,j] += 1/dt/dt
        # Space difference
        mat[i,j] += (c*c/dx/dx) * 2
        mat[i,j+1] += -(c*c/dx/dx)
        mat[i,j-1] += -(c*c/dx/dx)
        # Add the completed stencil
        eqs.append(mat)
        # Add the completed source
        col.append( f(t[i],x[j]) )

In [ ]:
# IVP Dirichlet
for j in range(0,N_x):
    mat = np.zeros((N_t, N_x))
    mat[0,j] = 1
    eqs.append(mat)
    col.append(g_0(x[j]))
# IVP Neumann
for j in range(1,N_x-1):
    mat = np.zeros((N_t, N_x))
    mat[0+1,j] = 1
    mat[0,j] = -1
    eqs.append(mat)
    col.append(g_1(x[j]))

In [ ]:
# Space bounds
for i in range(1,N_t):
    mat = np.zeros((N_t, N_x))
    mat[i,0] = 1
    eqs.append(mat)
    col.append(h_1)
for i in range(1,N_t):
    mat = np.zeros((N_t, N_x))
    mat[i,-1] = 1
    eqs.append(mat)
    col.append(h_2)

In [ ]:
# Reshape for matrix solving
eqs = np.array(eqs)
col = np.array(col)
print(eqs.shape, col.shape)
eqs = np.reshape(eqs,(N_t*N_x, N_t*N_x))
print(eqs.shape, col.shape)
# Solve the system
u = np.linalg.solve(eqs,col)
# Then reshape solution to the original
u = np.reshape(u, (N_t,N_x))

#### Visualize the Result

In [ ]:
fig, ax = plt.subplots()
ln, = plt.plot(x, u[0,:], animated=True)

def init():
    ln.set_data(x,u[0,:])
    return ln,

def update(frame_n):
    ln.set_data(x, u[frame_n,:])
    return ln,

t_anim = 5
anim = FuncAnimation(fig, update, frames=N_t, init_func=init, blit=True, interval = t_anim*1000/N_t, repeat=True)
plt.close()
HTML(anim.to_jshtml())